In [ ]:
!pip install fredapi

In [14]:
import pandas as pd
import json
import numpy as np
from fredapi import Fred
from tqdm import tqdm

from typing import Any

In [ ]:
import getpass
api_key = getpass.getpass(prompt='Please enter your FRED API Key: ')

In [17]:
def fred_series(series_id : str, value_col_name = 'VALUE') -> Any:
    fred = Fred(api_key=api_key)
    raw_data = fred.get_series(series_id)
    data = raw_data.reset_index()
    data = data.rename(columns={"index":"DATE", 0:value_col_name})
    data[value_col_name] = round(data[value_col_name], 2)
    return data

In [32]:
from collections import namedtuple

# Series - named tuple containing series ID, name of target value column, csv file name (without csv suffix)
Series = namedtuple('Series', ['id', 'value_col', 'file_name', 'data'], defaults=[None, None, None, None])

source = [Series(id='FEDFUNDS', value_col='EFFR', file_name='effr.csv'), 
          Series(id='UNRATE', value_col='UNEMPLOYMENT', file_name='unrate.csv'),
          Series(id='CPIAUCSL', value_col='CPI', file_name='cpi.csv'),
          Series(id='FPCPITOTLZGUSA', value_col='INFLATION', file_name='inflation.csv')
         ]
source

[Series(id='FEDFUNDS', value_col='EFFR', file_name='effr.csv', data=None),
 Series(id='UNRATE', value_col='UNEMPLOYMENT', file_name='unrate.csv', data=None),
 Series(id='CPIAUCSL', value_col='CPI', file_name='cpi.csv', data=None),
 Series(id='FPCPITOTLZGUSA', value_col='INFLATION', file_name='inflation.csv', data=None)]

In [33]:
target = []
for series in tqdm(source, "Fetching series data"):
    data = fred_series(series.id, series.value_col)
    target.append(Series(id=series.id, value_col=series.value_col, file_name=series.file_name, data=data))
 

Fetching series data: 100%|███████████████████████| 4/4 [00:01<00:00,  2.66it/s]


In [34]:
for series in tqdm(target, "Saving series data"):
    series.data.to_csv(series.file_name, index=False)

Saving series data: 100%|████████████████████████| 4/4 [00:00<00:00, 258.21it/s]
